# Proceso Gaussiano

* Entonces, los procesos gaussianos surgieron inicialmente cuando estábamos mirando la interpolación.
* Queríamos una forma de interpolar entre puntos pero con cierta incertidumbre sobre nuestro valor.
* Entonces, hagamos un resumen rápido de la interpolación

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal as mn
import seaborn as sb
cs = ['#56d870', '#f9ee4a', '#44d9ff', '#f95b4a', '#3d9fe2', '#ffa847', '#c4ef7a', '#e195e2', '#ced9ed', '#fff29b']

In [ ]:
def fn(xs):
    return np.exp((xs + 10.5)**0.1) + np.sin(xs) / (xs + 1) + np.cos(2.5 * xs**0.5)**2

xs = np.array([1, 2, 4, 5, 7, 9, 10])
ys = fn(xs)
x_fine = np.linspace(1, 10, 200)
y_fine = fn(x_fine)

plt.plot(xs, ys, 'o', label="Muestras", markersize=5)
plt.plot(x_fine, y_fine, label="Función", color="k", alpha=0.5, linestyle="-")
plt.legend();

In [ ]:
from scipy.interpolate import interp1d

y_linear = interp1d(xs, ys, kind="linear")(x_fine)
y_quad = interp1d(xs, ys, kind="quadratic")(x_fine)
y_cubic = interp1d(xs, ys, kind="cubic")(x_fine)

plt.plot(xs, ys, 'o', label="Muestras", markersize=5)
plt.plot(x_fine, y_fine, label="Funcion", color="w", ls="-", alpha=0.5)
plt.plot(x_fine, y_linear, label="Lineal")
plt.plot(x_fine, y_quad, label="Cuadratico")
plt.plot(x_fine, y_cubic, label="Cúbico")
plt.legend();

Así que muchas opciones de interpolación diferentes. Nada se parece a lo que queremos. Ahora, aparte, creemos una matriz de covarianza altamente correlacionada.

In [ ]:
x = np.arange(20)
length = 50
cov = np.exp(-(1 / length) * (x - np.atleast_2d(x).T)**2)
sb.heatmap(cov);

In [ ]:
for i in range(5):
    rvs = mn.rvs(cov=cov)
    plt.plot(x, rvs, 'o-', ms=5)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

rvs = mn.rvs(cov=cov, size=200)
rvs -= np.linspace(rvs[:, 0], rvs[:, -1], x.size).T  # Ayuda conceptual, no cómo lo harías realmente/

mean = rvs.mean(axis=0)
std = np.std(rvs, axis=0)

plt.plot(x, mean, "o-", ms=5)
plt.fill_between(x, mean + std, mean - std, alpha=0.4)
for i in range(100):
    plt.plot(x, rvs[i, :], ls="-", c="k", alpha=0.1, lw=0.5)


En el proceso de pseudo-GP anterior, queríamos interpolar 20 puntos (bueno, 18 puntos 'desconocidos' dado que los extremos son fijos), y lo hicimos dibujando desde un Gaussiano 20-D. Pero, obviamente, hay un número infinito de puntos a los que podríamos querer interpolar, por lo que dibujaríamos a partir de una Gaussiana de dimensión infinita. Esto es lo que significa "no paramétrico": dimensiones infinitas.

Y dibujando una y otra vez podemos obtener una predicción media y una desviación estándar. Tenga en cuenta que esto depende de nuestra elección de covarianza. El cov que hemos utilizado tiene un solo parámetro, su escala de longitud. Cuanto más grande sea $l$, más suaves y menos cambiantes serán las predicciones. Este número que 'escogemos' se conoce como hiperparámetro. Y es algo en lo que tenemos que encajar al crear un médico de cabecera real. Y en términos de nomenclatura, pasando de un proceso MVGaussiano a uno Gaussiano no tenemos una media, tenemos una función media (porque podemos pedir la media en cualquier x). De manera similar, la matriz de covarianza depende de los puntos de datos de entrada (en nuestro ejemplo anterior no teníamos ninguno, solo arreglamos el primer y el último punto) y los puntos en los que queremos evaluar, por lo que tenemos una función de covarianza. La forma de la covarianza (es decir, teníamos la distancia al cuadrado) se llama núcleo (puedes tener diferentes funciones, la distancia al cuadrado es útil)

### Procesos Gaussianos

Interpolación con ``esteroides``. Para esta sección, necesitará sklearn. `pip install sklearn` si no lo tiene.

El proceso gaussiano funciona entrenando un modelo, que se ajusta a los hiperparámetros del núcleo específico que proporciona. La dificultad está en saber qué kernel construir y luego dejar que el modelo entrene. Este núcleo esencialmente relaciona cómo cada punto de datos afecta las regiones en el espacio de parámetros.

Todo lo que dice este núcleo es que si está interpolando en el punto x, que se encuentra entre el punto A y el punto B, determina cuánto contribuye A (pero tomando $\exp(-(x-A)^2/(2\sigma)^2 )$) y luego haga lo mismo para B. Obtiene un peso diferente para A y B, y lo usa para calcular cuál cree que es el valor en x. La parte de entrenamiento de esto es encontrar el mejor valor de $\sigma$ en el núcleo que da los mejores resultados.

In [ ]:
plt.plot(xs, ys, 'o', label="Muestras", markersize=5)
plt.plot(x_fine, y_fine, label="Funcion", color="k", ls="-", alpha=0.5)
plt.legend();

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

kernels = [
    RBF(length_scale=1.0, length_scale_bounds=(1.0, 1.01)), 
    RBF(length_scale=2.0, length_scale_bounds=(2.0, 2.01)),
    RBF(length_scale=20.0, length_scale_bounds=(20.0, 20.01)),
]

fig, axes = plt.subplots(figsize=(10, 4), ncols=len(kernels), sharey=True)
fig.subplots_adjust(wspace=0, hspace=0)

for k, ax in zip(kernels, axes):
    gp = GaussianProcessRegressor(kernel=k)
    gp.fit(np.atleast_2d(xs).T, ys)
    
    y_mean, y_std = gp.predict(x_fine[:, None], return_std=True)

    ax.plot(xs, ys, 'o', label="Samples", markersize=5)
    ax.plot(x_fine, y_fine, label="Function", color="k", ls="-", lw=1, alpha=0.5)
    ax.plot(x_fine, y_mean)
    ax.fill_between(x_fine, y_mean + 2 * y_std, y_mean - 2 * y_std, alpha=0.5)
    ax.set_title(str(k) + f" {y_std.max():0.4f}")
    

Se Puede ver a la izquierda que tenemos un modelo donde la escala de longitud es demasiado pequeña. A la derecha, es tan grande que aleja incluso los puntos de datos conocidos de donde pertenecen. Pero el medio se ve bastante bien.

Lo que hacemos es ajustar los hiperparámetros para maximizar la probabilidad marginal condicionada a los hiperparámetros. No entraré en matemáticas, pero la probabilidad marginal tiene un término para qué tan bien se ajusta a los datos (por lo que la gráfica de la derecha tendría un puntaje horrible), un término de normalización y una penalización por complejidad (por lo que la gráfica de la mano izquierda modelo sería demasiado complejo porque su escala es demasiado pequeña y permite demasiada libertad).

Pero esto solo habla de cómo entrenar hiperparámetros. Hay un montón de núcleos diferentes entre los que se puede elegir, y esa es la tarea realmente difícil. Si se usa procesos gaussianos profundos, también puedes hacerlo, pero eso va mucho más allá de este curso.

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import ExpSineSquared as ES, RationalQuadratic as RQ

kernels = [
    C() * RBF(), 
    ES(periodicity=10),
    C() * RQ() + RBF()
]

fig, axes = plt.subplots(figsize=(10, 4), ncols=len(kernels), sharey=True)
fig.subplots_adjust(wspace=0, hspace=0)

for k, ax in zip(kernels, axes):
    gp = GaussianProcessRegressor(kernel=k)
    gp.fit(np.atleast_2d(xs).T, ys)
    
    y_mean, y_std = gp.predict(x_fine[:, None], return_std=True)

    ax.plot(xs, ys, 'o', label="Muestras", markersize=5)
    ax.plot(x_fine, y_fine, label="Funcion", color="k", ls="-", lw=1, alpha=0.5)
    ax.plot(x_fine, y_mean)
    ax.fill_between(x_fine, y_mean + 2 * y_std, y_mean - 2 * y_std, alpha=0.5)


In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

gp = GaussianProcessRegressor(kernel=C() * RBF())
gp.fit(np.atleast_2d(xs).T, ys)
y_pred, sigma = gp.predict(np.atleast_2d(x_fine).T, return_std=True)

upper, lower = y_pred + 1.96 * sigma, y_pred - 1.96 * sigma

plt.plot(xs, ys, 'o', label="Muestras", markersize=5)
plt.plot(x_fine, y_fine, label="Funcion", color="k", ls="-", alpha=0.5)
plt.plot(x_fine, y_pred, label="GP")
plt.fill_between(x_fine, upper, lower, alpha=0.5, label="95% confianza")
plt.legend();

### Procesos Gaussianos con incertidumbre

Conceptualmente, ¿qué pasa si nuestros puntos de datos tienen incertidumbre? Lo que estamos haciendo es agregar una matriz de covarianza adicional a la mezcla, donde para nuestra incertidumbre independiente será una matriz diagonal. Formalmente, esto se llama regularización de Tikhonov.

In [ ]:
np.random.seed(2)

xs2 = np.array([1, 1.5, 2, 3, 4, 5, 7, 9, 10])
ys2 = fn(xs2)

err_scale = np.random.uniform(low=0.03, high=0.1, size=ys2.shape)
err = np.random.normal(loc=0, scale=err_scale, size=ys2.shape)
ys_err = ys2 + err

plt.errorbar(xs2, ys_err, yerr=err, fmt="o", label="Muestras", markersize=5)
plt.plot(x_fine, y_fine, label="Funcion", color="k", alpha=0.5, linestyle="-")
plt.legend();

In [ ]:
gp = GaussianProcessRegressor(alpha=err**2)
gp.fit(np.atleast_2d(xs2).T, ys_err)
y_pred, sigma = gp.predict(np.atleast_2d(x_fine).T, return_std=True)

upper, lower = y_pred + 1.96 * sigma, y_pred - 1.96 * sigma

plt.errorbar(xs2, ys_err, yerr=err, fmt="o", label="Muestras", color="#f95b4a", markersize=5)
plt.plot(x_fine, y_fine, label="Funcion", color="k", ls=":", alpha=0.5)
plt.plot(x_fine, y_pred, label="GP", color="#44d9ff", ls="-")
plt.fill_between(x_fine, upper, lower, alpha=0.2, color="#44d9ff", label="95% confidence")
plt.legend();

Los procesos gaussianos requieren sutileza en el entrenamiento, especialmente en el decapado del kernel correcto. Es un campo muy profundo y rico en el que fácilmente puede pasar todo un doctorado investigando.